In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import asf_search as asf
from collections import defaultdict
import datetime
import pickle
from tqdm import tqdm
from functools import reduce
import itertools
from shapely import Point, LineString, Polygon, MultiPolygon
import operator
import lxml
from pykml.factory import KML_ElementMaker as KML
import cartopy

In [2]:
#Extracts data from the dataset ds within the time_filter (tuple or timespan) interval for the 
#variable var_name found in the deph range deph_range in meters, positive is under water, negative above water
#It can be either a tuble (min,max) or a value it needs to equal
#Quality controll is made for position, deph, time, and the variable
#Note depth is the coordinate index while deph (without t) is the actual depth in meters 
def valid_data_extraction(ds, time_filter, var_name, deph_range):
    if var_name not in ds.data_vars:
        raise ValueError(var_name, ' Not found')

    #Add longitude, latidude and position_qc as variables indexed by time,depth as all other variables
    TIME = ds['TIME'].values
    DEPTH = ds['DEPTH'].values
    n_DEPTHS = len(DEPTH)

    dataset_columns = {
        'LONG':ds['LONGITUDE'],
        'LAT':ds['LATITUDE'],
        'POS_QC':ds['POSITION_QC'],
    }

    ds_pos = xr.Dataset(
        data_vars=
        {k:(
            ["TIME", 'DEPTH'],
            np.repeat(np.reshape(v.values, (-1,1)), n_DEPTHS, axis=1),
            v.attrs,
        )for (k,v) in dataset_columns.items()},
        coords=dict(
            TIME=TIME,
            DEPTH=DEPTH,
        )
    ).drop_vars('DEPTH')
    ds = xr.merge([ds.drop_dims(['LATITUDE', 'LONGITUDE', 'POSITION']), ds_pos])
    
    #Filter for time of interest
    if type(time_filter) is tuple:
        ds = ds.sel(TIME=slice(time_filter[0], time_filter[1]))
    else:
        ds = ds.sel(TIME=time_filter)
    
    #Filter only avalible columns
    colum_names = [var_name]
    colum_names_qc = [var_name + '_QC']
    
    #Add fixed columns
    colum_names.extend(['LONG', 'LAT', 'DEPH'])
    colum_names_qc.extend(['DEPH_QC'])
    time_pos_qc = ['TIME_QC', 'POS_QC']
    
    #Filter for columns of interest
    ds = ds[colum_names + colum_names_qc + time_pos_qc]

    df = ds.to_dataframe()
    
    QC_good = [1.0, 7.0]
    #QC control for time and pos uses all of these values according to https://doi.org/10.13155/59938
    QC_time_pos_good = [1, 2, 5, 7, 8]
    
    #Filter the variable and depth for good quality data 
    filter_qc = [df[c_qc].isin(QC_good) for c_qc in colum_names_qc]
    #Filter for good time and pos 
    filter_qc.extend([df[c_qc].isin(QC_time_pos_good) for c_qc in time_pos_qc])
    #Add filter for deph value
    if type(deph_range) == tuple:
        filter_qc.append((deph_range[0] <= df['DEPH']) & (df['DEPH'] <= deph_range[1]))
    else:
        filter_qc.append(df['DEPH'] == deph_range)
    #Element-wise AND the filter
    filter_qc = reduce(operator.and_, filter_qc)
    df = df[filter_qc][colum_names]

    #Remove any duplicated measurements on different depths
    #To do so, first remove the depth from the index and then filter for unique time index by keeing the first
    df = df.reset_index('DEPTH')
    df = df[~df.index.duplicated(keep='first')]

    return df

In [3]:
data_dir = '/data/exjobb/sarssw/bouy/INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030/MO'

start_date = '2000-01-01'
end_date = '2021-12-31'
variables = ['VHM0', 'VAVH', 'WSPD']#['VHM0', 'VAVH', 'WSPD']

files = os.listdir(data_dir)
test_ds = xr.open_dataset(os.path.join(data_dir, 'GL_TS_MO_44005.nc')) #IR_TS_MO_6201039
test_ds

<xarray.Dataset>
Dimensions:       (TIME: 304956, LATITUDE: 304956, LONGITUDE: 304956,
                   POSITION: 304956, DEPTH: 5)
Coordinates:
  * TIME          (TIME) datetime64[ns] 1978-12-16T12:00:00 ... 2023-01-31T23...
  * LATITUDE      (LATITUDE) float32 43.18 43.18 43.18 43.18 ... 43.2 43.2 43.2
  * LONGITUDE     (LONGITUDE) float32 -69.18 -69.18 -69.18 ... -69.13 -69.13
Dimensions without coordinates: POSITION, DEPTH
Data variables: (12/29)
    TIME_QC       (TIME) float32 ...
    POSITION_QC   (POSITION) float32 ...
    DC_REFERENCE  (TIME) object ...
    DEPH          (TIME, DEPTH) float32 ...
    DEPH_QC       (TIME, DEPTH) float32 ...
    TEMP          (TIME, DEPTH) float64 ...
    ...            ...
    VTPK          (TIME, DEPTH) float64 ...
    VTPK_QC       (TIME, DEPTH) float32 ...
    WDIR          (TIME, DEPTH) float64 ...
    WDIR_QC       (TIME, DEPTH) float32 ...
    WSPD          (TIME, DEPTH) float64 ...
    WSPD_QC       (TIME, DEPTH) float32 ...
Attributes: (12/49)
    data_type:                      OceanSITES time-series data
    format_version:                 1.4
    platform_code:                  44005
    institution:                    National Data Buoy Center (NDBC) ; Nation...
    institution_edmo_code:          1463
    site_code:                       
    ...                             ...
    last_date_observation:          2023-01-31T23:50:00Z
    last_latitude_observation:      43.20100
    last_longitude_observation:     -69.12800
    date_update:                    2023-02-07T19:28:46Z
    history:                        2023-02-07T19:28:46Z : Creation
    data_mode:                      R

In [4]:
df = valid_data_extraction(test_ds, (start_date, end_date), 'WSPD', (-30,0))
df

,DEPTH,WSPD,LONG,LAT,DEPH
TIME,,,,,
2000-01-01 00:00:00.000000000,0,3.199,-69.180000,43.180,-4.9
2000-01-01 01:00:00.000000000,0,2.999,-69.180000,43.180,-4.9
2000-01-01 02:00:00.000000000,0,2.999,-69.180000,43.180,-4.9
2000-01-01 03:00:00.000000000,0,2.699,-69.180000,43.180,-4.9
2000-01-01 04:00:00.000000000,0,1.999,-69.180000,43.180,-4.9
...,...,...,...,...,...
2021-12-31 19:50:00.000000256,0,1.099,-69.126999,43.201,-4.9
2021-12-31 20:50:00.000000000,0,1.199,-69.126999,43.201,-4.9
2021-12-31 21:50:00.000000000,0,2.199,-69.126999,43.201,-4.9


In [5]:
# Searches for SAR images that overlaps with the data in file according to the filters specified by the arguments:

# file: specifies the file name of the bouy data

# data_dir: is the directory

# start_date & end_date: are the time filters for the data

# variables_dephs: The keys are the variable to extract data for while the values are the accepted depth ranges 
# (in meters) for the corresponding variable negative values are depth under surface while positive is above 

# max_time_diff_s: Is the absolute time difference between image and bouy observation in seconds

# land_multipolygon: is the shapely multipolygon object that describes the land and close to shore map of earth
# observations from these areas are filtered

# result_df: is the dataframe where the result is saved, if it is passed as None it is automatically created with the correct columns

# The data is also quiality controlled and only rows containing valid in regards to the variable, time, position and depth are accepted
def search_file(file, data_dir, start_date, end_date, variables_dephs, max_time_diff_s, land_multipolygon, result_df):
    #Conditionally create the result dataframe
    if result_df is None:
        result_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {
            'bouy_file_name':str,
            'bouy_longitude':float,
            'bouy_latitude':float,
            'bouy_time':np.dtype('<M8[ns]'), #np.datetime64
            'bouy_depth_index':int, #The depth index
            'bouy_depth':float, #The deph (depth) value in meters, positive is under water
            'bouy_variable_name':str,
            'bouy_variable_value':float,
            'sar_url':str,
            'sar_mode':str,
            'sar_polarization':str,
            'sar_platform':str,
            'sar_start_time':np.dtype('<M8[ns]'), #np.datetime64
            'sar_stop_time':np.dtype('<M8[ns]'), #np.datetime64
            'sar_coordinates':object, #List
        }.items()})

    #Load the data from the file
    file_path = os.path.join(data_dir, file)
    xar = xr.open_dataset(file_path)# , engine='scipy')
    xar_vars = list(xar.data_vars)

    #Clear any eventual prior asf search result
    asf_results = None
    
    #Filter for variables that exist in the data
    common_variables = set(variables_dephs.keys()).intersection(xar_vars)
    
    if len(common_variables) == 0:
        print(file + " does not have any of the variables " + str(variables))
        return result_df
    
    for var in common_variables:
        try:
            #extract dataframe of the correct timeinterval and variable 
            df = valid_data_extraction(xar, (start_date, end_date), var, variables_dephs[var])
        except Exception as e:
            print(file, ': data extraction failed with:', e)
            continue
        
        if df.empty:
            print(file + " does not have any valid data for this timeperiod")
            continue
            
        #Only search once (assume similar results in regards to min and max longitude and latitude for each variable)
        if asf_results is None:
            #extract long and latitude
            long_min, lat_min = df[['LONG', 'LAT']].min()
            long_max, lat_max = df[['LONG', 'LAT']].max()

            #Create geographical search restriction
            coord_points = list(itertools.product(set([long_min, long_max]), set([lat_min, lat_max])))

            #For each possible length of coord_points set the prefix ans dufix for the asf search
            #Also create a shapely geometry to check for overlap with the land_multipolygon
            if len(coord_points) == 1:
                geo_prefix = 'POINT('
                geo_suffix = ')'
                geo_obj = Point(*coord_points)
            elif len(coord_points) == 2:
                #raise Exception("First time we see only changes in one coordinate, debug and make sure it works porpery")
                geo_prefix = 'LINESTRING('
                geo_suffix = ')'
                geo_obj = LineString(coord_points)
            elif len(coord_points) == 4:
                #Double parenthesis was given from asf website export search as python function
                geo_prefix = 'POLYGON(('
                geo_suffix = '))'
                #Untangle the polygon itersection
                coord_points[2:4]=reversed(coord_points[2:4])
                #Form closed polygon by adding adding fist point as last
                coord_points.append(coord_points[0])
                geo_obj = Polygon(coord_points)

            #The area that of the locations in the data that is not near the shoreline
            off_shore = geo_obj.difference(land_multipolygon)

            #Check if the whole area of interest is close to shore or overlapps with land
            #in that case return
            if off_shore.is_empty:
                print(file + " is too close to land")
                return result_df

            #Filter data of df to only include data that does not lie close to shore
            #allow a small distance since points on the edge doeas not overlap according to shapely.overlaps 
            df_shore_filter =  df.apply(lambda row: Point([row['LONG'], row['LAT']]).distance(off_shore), axis=1) <= 0.00001
            df = df[df_shore_filter]

            geo_limit = ','.join([str(long) + ' ' + str(lat) for long,lat in coord_points])
            geo_limit = geo_prefix + geo_limit + geo_suffix
            
            print(geo_limit)

            #Search asf
            options = {
                'intersectsWith': geo_limit,
                'platform': 'SENTINEL-1',
                'instrument': 'C-SAR',
                'start': start_date,
                'end': end_date,
                'processingLevel': [
                    'GRD_HD',
                    'GRD_MD',
                    'GRD_MS',
                    'GRD_HS'
                ],
                'beamSwath': [
                    'IW',
                    'EW'
                ],
            }

            asf_results = asf.search(**options)
            print(len(asf_results), ' results found')
        
        for asf_result in asf_results:
            #Find mean time of image generation
            asf_result_start = datetime.datetime.fromisoformat(asf_result.properties['startTime'][:-5])
            asf_result_stop = datetime.datetime.fromisoformat(asf_result.properties['stopTime'][:-5])
            asf_result_mean_time = asf_result_start + (asf_result_stop-asf_result_start)/2
        
            #Check the shape type
            if asf_result.geometry['type'] != 'Polygon':
                raise ValueError('The shape of the search result is not Polygon but instead: ' + asf_result.geometry['type'])
            
            #Filter for points that lie within the image
            if geo_prefix != 'POINT(':
                polygon = Polygon(asf_result.geometry['coordinates'][0])
                df_dist_filter =  df.apply(lambda row: Point([row['LONG'], row['LAT']]).distance(polygon), axis=1) <= 0.0001
                df_close = df[df_dist_filter]
                
                #Skip is we have no overlaping points
                if df_close.empty:
                    #print("No geographical overlap")
                    continue
                    
            #No geographical overlap check needed (point search)
            else:
                df_close = df
            
            #Find closest entry in time
            closest_data_row = df_close.iloc[df_close.index.get_indexer([asf_result_mean_time], method='nearest')] #only one row of data

            max_time_diff = datetime.timedelta(seconds=max_time_diff_s)
            
            #Check if it is close enough to be considered overlapping
            if abs(closest_data_row.index[0] - asf_result_mean_time) <= max_time_diff:
                #Save result in the dataframe
                result_df.loc[len(result_df.index)] = [
                    file, #'bouy_file_name':str,
                    closest_data_row['LONG'][0], #'bouy_longitude':float,
                    closest_data_row['LAT'][0], #'bouy_latitude':float,
                    closest_data_row.index[0], #'bouy_time':np.dtype('<M8[ns]'), #np.datetime64
                    closest_data_row['DEPTH'][0], #'bouy_depth_index':int, #The depth index
                    closest_data_row['DEPH'][0], #'bouy_depth':float,
                    var, #'bouy_variable_name':str,
                    closest_data_row[var][0], #'bouy_variable_value':float,
                    asf_result.properties['url'], #'sar_url':str,
                    asf_result.properties['beamModeType'], #'sar_mode':str,
                    asf_result.properties['polarization'].split('+'), #'sar_polarization':str,
                    asf_result.properties['platform'], #'sar_platform':str,
                    asf_result_start, #'sar_start_time':np.dtype('<M8[ns]'), #np.datetime64
                    asf_result_stop, #'sar_stop_time':np.dtype('<M8[ns]'), #np.datetime64
                    asf_result.geometry['coordinates'][0], #'sar_coordinates':list,
                ]
                
            else:
                #print('No time overlap')
                pass
            
        
    return result_df

In [6]:
data_dir = '/data/exjobb/sarssw/bouy/INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030/MO'
start_date = '2021-01-01'
end_date = '2021-12-31'
variables_deph = {'VHM0':0, 'VAVH':0, 'WSPD':(-30,0)}#['VHM0', 'VAVH', 'WSPD']
max_time_diff_s = 60*60

write_folder = './1h_survey'
result_df_fn = 'result_df'
kml_pinmap_fn = 'kml_pinmap'

#Dataframe where the results is saved, automatically created in search_file
result_df = None

#Load and create land multipolygon, buffered (expanded) to limit distance to shore
land_list = list(cartopy.feature.NaturalEarthFeature('physical', 'land', '50m').geometries())
polygon_list = []
for p  in land_list:
    if type(p) == MultiPolygon:
        polygon_list.extend(p.geoms)
    else:
        polygon_list.append(p)
land_multipolygon = MultiPolygon([p.buffer(0.01) for p in polygon_list])

# Create search_file function with only necessary parameters for the file loop below
def search_file_fixed_params(file, result_df):
    return search_file(file, data_dir, start_date, end_date, variables_deph, max_time_diff_s, land_multipolygon, result_df)

files = set(os.listdir(data_dir))

file_filter = [
    'GL_TS_MO_41121.nc', #Flips longitude sign in the middle of the data, from 66 to -66???! resutlts in asf search with over 7000 matches.
]
files = set(files).difference(file_filter)

for file in tqdm(list(files)):
    print('\n',file)
    result_df = search_file_fixed_params(file, result_df)

  0%|          | 0/2487 [00:00<?, ?it/s]


 GL_WS_MO_46041.nc
GL_WS_MO_46041.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']

 GL_TS_MO_41159.nc
POINT(-76.9489974975586 34.2130012512207)


  0%|          | 2/2487 [00:08<3:03:50,  4.44s/it]

27  results found

 GL_TS_MO_48011.nc


  0%|          | 5/2487 [00:08<59:24,  1.44s/it]  

...

100%|█████████▉| 2484/2487 [3:50:47<00:15,  5.22s/it]

GL_TS_MO_52073.nc does not have any valid data for this timeperiod

 GL_TS_MO_MEDS190.nc
GL_TS_MO_MEDS190.nc does not have any valid data for this timeperiod

 GL_WS_MO_44097.nc
GL_WS_MO_44097.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']

 GL_TS_MO_45168.nc


100%|█████████▉| 2486/2487 [3:50:47<00:03,  3.58s/it]

GL_TS_MO_45168.nc is too close to land

 GL_TS_MO_41025.nc
POLYGON((-75.4540023803711 35.025001525878906,-75.4540023803711 35.0099983215332,-75.36299896240234 35.0099983215332,-75.36299896240234 35.025001525878906,-75.4540023803711 35.025001525878906))
57  results found


100%|██████████| 2487/2487 [3:52:31<00:00,  5.61s/it]


In [7]:
result_df

,bouy_file_name,bouy_longitude,bouy_latitude,bouy_time,bouy_depth_index,bouy_depth,bouy_variable_name,bouy_variable_value,sar_url,sar_mode,sar_polarization,sar_platform,sar_start_time,sar_stop_time,sar_coordinates
0,GL_TS_MO_41159.nc,-76.948997,34.213001,2021-12-27 22:56:00,0,0.0,VHM0,1.109000,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-27 23:05:44,2021-12-27 23:06:09,"[[-77.787575, 34.208473], [-77.43821, 32.70319..."
1,GL_TS_MO_41159.nc,-76.948997,34.213001,2021-12-15 22:56:00,0,0.0,VHM0,0.849000,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-15 23:05:45,2021-12-15 23:06:10,"[[-77.787628, 34.208488], [-77.438286, 32.7032..."
2,GL_TS_MO_41159.nc,-76.948997,34.213001,2021-12-03 22:56:00,0,0.0,VHM0,0.809000,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-03 23:05:46,2021-12-03 23:06:11,"[[-77.787346, 34.208439], [-77.438026, 32.7031..."
3,GL_TS_MO_41159.nc,-76.948997,34.213001,2021-11-21 22:56:00,0,0.0,VHM0,1.249000,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-11-21 23:05:46,2021-11-21 23:06:11,"[[-77.787842, 34.208469], [-77.438553, 32.7032..."
4,GL_TS_MO_41159.nc,-76.948997,34.213001,2021-11-09 22:56:00,0,0.0,VHM0,1.199000,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-11-09 23:05:46,2021-11-09 23:06:11,"[[-77.7882, 34.208534], [-77.438843, 32.703255..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79879,GL_TS_MO_41025.nc,-75.362999,35.025002,2021-01-25 23:10:00,0,-4.1,WSPD,5.499000,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-01-25 23:06:03,2021-01-25 23:06:28,"[[-78.142181, 35.713089], [-77.787048, 34.2084..."
79880,GL_TS_MO_41025.nc,-75.362999,35.025002,2021-01-20 23:00:00,0,-4.1,WSPD,7.999000,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-01-20 22:57:57,2021-01-20 22:58:26,"[[-76.225327, 36.428345], [-75.809944, 34.6814..."
79881,GL_TS_MO_41025.nc,-75.362999,35.025002,2021-01-13 23:10:00,0,-4.1,WSPD,3.299000,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-01-13 23:06:03,2021-01-13 23:06:28,"[[-78.142395, 35.7131], [-77.78727, 34.208504]..."
79882,GL_TS_MO_41025.nc,-75.362999,35.025002,2021-01-08 23:00:00,0,-4.1,WSPD,15.499001,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-01-08 22:57:57,2021-01-08 22:58:26,"[[-76.225426, 36.428337], [-75.810219, 34.6821..."


In [8]:
#Print the average wind speed and number of measurements grouped by depth 
result_df[result_df['bouy_variable_name'] == 'WSPD'][['bouy_depth', 'bouy_variable_value']].groupby('bouy_depth').agg(['mean', 'count'])

bouy_variable_value      
                          mean count
bouy_depth                          
-28.00               11.279556    72
-14.00                7.517844   461
-12.00                6.693656   221
-10.00                7.401830  2039
-9.00                 8.879128    47
-5.70                 7.614184   191
-5.00                 5.851424   250
-4.90                 6.276298   370
-4.70                 6.742138    51
-4.60                 7.503030   273
-4.50                 5.656634   786
-4.10                 6.128891  1954
-4.00                 6.118517  1404
-3.80                 6.429544  1601
-3.60                 5.641857     7
-3.50                 5.674610    41
-3.20                 6.790097  1178
-3.10                 5.386500    16
-3.00                 5.397063  3386
-2.95                 5.434000    40
-2.60                 7.453546    11
-2.00                 4.286976   608
 0.00                 6.926007  7130

In [9]:
#Save resuld_df with pickle

#Conditionally creates the folder for the result
os.makedirs(write_folder, exist_ok=True)

with open(os.path.join(write_folder, result_df_fn),'wb') as f_w:
    pickle.dump(result_df,f_w)


In [10]:
#Save KML pin maps, one for each variable

#Total number of datapoints per variable type
var_total = result_df['bouy_variable_name'].value_counts()
 
#pandas series for count of unique variable name, file name pairs
var_file_count = list(result_df.value_counts(subset=['bouy_variable_name', 'bouy_file_name']).items())

#dataframe for coordinates of unique variable name, file name pairs
var_file_coord = (result_df.groupby(['bouy_variable_name', 'bouy_file_name'])[['bouy_longitude', 'bouy_latitude']].first())


#Create separate kml maps for each variable name
KML_fldrs = {}
for var_name in np.unique(result_df['bouy_variable_name']):
    KML_fldrs[var_name] = KML.Folder(
        KML.name(var_name + " " + write_folder[write_folder.find('/')+1:]),
        KML.description('Total datapoints: ' + str(var_total[var_name])),
    )

#iterate over all unique variable name, file name pairs
for ((var_name, file_name), count) in var_file_count:
    #extract longitude and latitude
    long, lat = var_file_coord.loc[(var_name, file_name)][['bouy_longitude', 'bouy_latitude']]
    
    #Create the pin
    pin = KML.Placemark(
        KML.name(str(count)),
        KML.description(file_name),
        KML.Point(
            KML.coordinates(str(long) + "," + str(lat))
        )
    )
    KML_fldrs[var_name].append(pin)
    
for var_name, KML_fld in KML_fldrs.items():
    with open(os.path.join(write_folder, kml_pinmap_fn + '_' + var_name + '.kml'), 'w') as f_w:
        f_w.write(lxml.etree.tostring(KML_fld, pretty_print=True).decode())

In [ ]:
#Save KML pin map if we have all the variables ['VHM0', 'VAVH', 'WSPD']
if all([v in variables for v in ['VHM0', 'VAVH', 'WSPD']]):
    #Number of  overlaps
    mult_var = result_df.groupby(['sar_url', 'bouy_file_name', 'bouy_longitude', 'bouy_latitude'])[['bouy_variable_name']].aggregate(lambda tdf: tdf.unique().tolist())
    mult_var['len'] = mult_var['bouy_variable_name'].apply(len)
    mult_var['contains_wind'] = mult_var['bouy_variable_name'].apply(lambda x: 'WSPD' in x)
    mult_var['also_VAVH'] = mult_var['bouy_variable_name'].apply(lambda x: 'VAVH' in x)
    mult_var['also_VHM0'] = mult_var['bouy_variable_name'].apply(lambda x: 'VHM0' in x)
    mult_var = mult_var[(mult_var['len'] > 1) & mult_var['contains_wind']].drop(labels='contains_wind', axis=1)

    wind_and_VAVH = mult_var[mult_var['also_VAVH']].drop(labels=['also_VAVH', 'also_VHM0'], axis=1).groupby(['bouy_file_name', 'bouy_longitude', 'bouy_latitude'])['len'].count()
    wind_and_VHM0 = mult_var[mult_var['also_VHM0']].drop(labels=['also_VAVH', 'also_VHM0'], axis=1).groupby(['bouy_file_name', 'bouy_longitude', 'bouy_latitude'])['len'].count()

    #Save overlapping wind and wave info to KML maps
    for kml_pinmap_fn, data_var in [('WSPD_and_VHM0', wind_and_VHM0), ('WSPD_and_VAVH', wind_and_VAVH)]:
        KML_fldr = KML.Folder(
            KML.name(kml_pinmap_fn + " " + write_folder[write_folder.find('/')+1:]),
            KML.description('Total datapoints: ' + str(data_var.sum())),
        )

        #iterate over all unique variable name, file name pairs
        for ((file_name, long, lat), count) in data_var.items():
            #Create the pin
            pin = KML.Placemark(
                KML.name(str(count)),
                KML.description(file_name),
                KML.Point(
                    KML.coordinates(str(long) + "," + str(lat))
                )
            )
            KML_fldr.append(pin)

        with open(os.path.join(write_folder, kml_pinmap_fn + '_' + var_name + '.kml'), 'w') as f_w:
            f_w.write(lxml.etree.tostring(KML_fldr, pretty_print=True).decode())